# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge conventions which are expected by EmbedTrack.

In [1]:
import os

import numpy as np
from tifffile import imwrite

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.isbi_utils import trk_to_isbi
from deepcell_tracking.trk_io import load_trks

In [2]:
source_data = '../../data/test.trks'
data_dir = 'data'
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(raw_dir, 'Analysis/Segmentation_GT')
seg_dc_dir = os.path.join(raw_dir, 'Analysis/Segmentation_DeepCell')
gt_dir = os.path.join(data_dir, 'GT')

for d in [raw_dir, seg_gt_dir, seg_dc_dir, gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [3]:
data = load_trks(source_data)

Load the DeepCell nuclear segmentation model to test the algorithm on predicted instead of ground truth segmentations

In [4]:
app = NuclearSegmentation()

2022-11-09 00:37:44.248981: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 00:37:44.915438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1


In [5]:
def predict_without_zeros(app, X):
    """Remove zero padding to avoid adverse effects on model performance"""
    # Calculate position of padding based on first frame
    # Assume that padding is in blocks on the edges of image
    good_rows = np.where(X.any(axis=0))[0]
    good_cols = np.where(X.any(axis=1))[0]

    slc = (
        slice(good_cols[0], good_cols[-1] + 1),
        slice(good_rows[0], good_rows[-1] + 1),
    )

    y_pred = np.zeros_like(X)
    y_pred[slc] = app.predict(X[slc])

    return y_pred

Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will test.

In [6]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    gt_subdir = os.path.join(gt_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))
    seg_dc_subdir = os.path.join(seg_dc_dir, '{:03}'.format(batch_no + 1))
    
    # Create directories if needed
    for d in (raw_subdir, gt_subdir, seg_gt_subdir, seg_dc_subdir):
        if not os.path.exists(d):
            os.makedirs(d)
                
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineages = data['lineages'][batch_no]
    
    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(gt_subdir, 'man_track.txt')
    df = trk_to_isbi(lineages)
    df.to_csv(text_file, sep=' ', header=False)
    
    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    # We assume here that the empty frames are at the end of the movie (padding rather than skipped)
    movie_len = len(good_frames)
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(movie_len):
        name_raw = os.path.join(raw_subdir, 't{:03}.tif'.format(i))
        name_seg_gt = os.path.join(seg_gt_subdir, 't{:03}.tif'.format(i))
        name_seg_dc = os.path.join(seg_dc_subdir, 't{:03}.tif'.format(i))
        name_gt = os.path.join(gt_subdir, 't{:03}.tif'.format(i))
        
        # Generate deepcell prediction
        pred = predict_without_zeros(app, X[i:i+1, ..., channel:channel+1])
        
        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_seg_gt, y[i, ..., channel].astype('uint16'))
        imwrite(name_seg_dc, pred.astype('uint16'))
        imwrite(name_gt, y[i, ..., channel].astype('uint16'))

2022-11-09 00:37:57.263247: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima

# 2. Tracking

The steps for tracking will be performed in Matlab and the Baxter Algorithm GUI. You will need access to Matlab 2019b. The instructions below outline the general workflow for generating benchmarking results using the Baxter algorithm. However additional support for using the software is available in the user guide located in the KTH-SE folder (`KTH-SE/UserGuide/UserGuide.pdf`).

## Part 1 - Setup
1. Open the KTH-SE folder in Matlab
2. Open `BaxterAlgorithms.m` in the Matlab editor and then hit Run. This should open a GUI.
3. Within the GUI, select `Files > Open experiment > Browse...`. Navigate to the `data/raw` folder located within this Baxter folder.
4. Ensure that all files are selected in the list on the right hand side of the GUI.
## Part 2 - GT Segmentations
5. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationGT.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "GT" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "GT" version for export.
## Part 3 - DeepCell Segmentations
8. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationDeepCell.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "DeepCell" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "DeepCell" version for export.


# 3. Evaluation

In [1]:
import os

import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [2]:
data_dir = 'data/GT'
export_gt_dir = 'data/raw/Analysis/CellDataGT/RES'
export_dc_dir = 'data/raw/Analysis/CellDataDeepCell/RES'

data_ids = os.listdir(data_dir)

node_match_threshold = 0.6

In [7]:
benchmarks = []

for results_dir, s in zip([export_gt_dir, export_dc_dir], ['GT', 'Deepcell']):
    for data_id in data_ids:
        gt_dir = os.path.join(data_dir, f'{data_id}')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir)
        benchmarks.append({
            'model': f'Baxter - {s}',
            'data_id': data_id,
            **m.stats
        })
        
df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 21_42 division completely
missed node 37_35 division completely
missed node 38_5 division completely
missed node 43_4 division completely
missed node 50_42 division completely
missed node 5_6 division completely
missed node 10_66 division completely
missed node 15_66 division completely
missed node 25_57 division completely
30_37 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 55_27 division completely
missed node 76_44 division completely
missed node 94_36 division completely
missed node 104_34 division completely
missed node 112_6 division completely
missed node 118_0 division completely
missed node 121_42 division completely
missed node 125_22 division completely
missed node 127_51 division completely
missed node 128_43 division completely
missed node 140_10 division completely
missed node 144_28 division completely
missed node 147_33 division completely
missed node 160_64 division completely
missed node 163_66 division completely
missed node 167